In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Set the path to your ZIP file
zip_file_path = '/content/Soil.zip'

# Set the class names
class_names = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy loam', 'Sandy soil']

# Set the desired image size
image_size = (224, 224)

# Create lists to store the data and labels
data = []
labels = []

# Read the images from the ZIP file and preprocess them
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    soil_types_folder = 'Soil Types'
    for class_name in class_names:
        class_dir = os.path.join(soil_types_folder, class_name)
        for file_name in zip_ref.namelist():
            if file_name.startswith(class_dir) and not file_name.endswith('/'):
                with zip_ref.open(file_name) as image_file:
                    image = Image.open(image_file)
                    # Resize the image to the desired size
                    image = image.resize(image_size)
                    # Convert the image to grayscale
                    image = image.convert('L')
                    # Perform any necessary additional preprocessing on the image here
                    # ...
                    # Convert the image to a NumPy array
                    image = np.array(image)
                    # Add the preprocessed image to the data list
                    data.append(image)
                    # Add the corresponding label to the labels list
                    labels.append(class_name)

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the pixel values to the range [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=train_data[0].shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(class_names), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_data, val_labels)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Set the path to your ZIP file
zip_file_path = '/content/Soil.zip'

# Set the class names
class_names = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy loam', 'Sandy soil']

# Set the desired image size
image_size = (224, 224)

# Create lists to store the data and labels
data = []
labels = []

# Read the images from the ZIP file and preprocess them
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    soil_types_folder = 'Soil Types'
    for class_name in class_names:
        class_dir = os.path.join(soil_types_folder, class_name)
        for file_name in zip_ref.namelist():
            if file_name.startswith(class_dir) and not file_name.endswith('/'):
                with zip_ref.open(file_name) as image_file:
                    image = Image.open(image_file)
                    # Resize the image to the desired size
                    image = image.resize(image_size)
                    # Convert the image to grayscale
                    image = image.convert('L')
                    # Perform any necessary additional preprocessing on the image here
                    # ...
                    # Convert the image to a NumPy array
                    image = np.array(image)
                    # Add the preprocessed image to the data list
                    data.append(image)
                    # Add the corresponding label to the labels list
                    labels.append(class_name)

# Convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Perform label encoding to convert class names to numeric labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Normalize the pixel values of the images
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

# Convert grayscale images to expected input shape (add channel dimension)
train_data = np.expand_dims(train_data, axis=-1)
test_data = np.expand_dims(test_data, axis=-1)

# Create TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels))

# Batch and shuffle the datasets
batch_size = 64
train_ds = train_ds.batch(batch_size).shuffle(len(train_data))
test_ds = test_ds.batch(batch_size)

# Define your model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(train_ds, validation_data=test_ds, epochs=epochs)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data, test_labels)

# Print the accuracy
print('Test accuracy:', accuracy)


In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

# Set the path to your ZIP file
zip_file_path = '/content/Soil.zip'

# Set the class names
class_names = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy loam', 'Sandy soil']

# Set the desired image size
image_size = (224, 224)

# Create lists to store the data and labels
data = []
labels = []

# Read the images from the ZIP file and preprocess them
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    soil_types_folder = 'Soil Types'
    for class_name in class_names:
        class_dir = os.path.join(soil_types_folder, class_name)
        for file_name in zip_ref.namelist():
            if file_name.startswith(class_dir) and not file_name.endswith('/'):
                with zip_ref.open(file_name) as image_file:
                    image = Image.open(image_file)
                    # Resize the image to the desired size
                    image = image.resize(image_size)
                    # Convert the image to grayscale
                    image = image.convert('L')
                    # Perform any necessary additional preprocessing on the image here
                    # ...
                    # Convert the image to a NumPy array
                    image = np.array(image)
                    # Add the preprocessed image to the data list
                    data.append(image)
                    # Add the corresponding label to the labels list
                    labels.append(class_name)

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Add channel dimension to the data
data = np.expand_dims(data, axis=-1)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the pixel values to the range [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# Define the model architecture
def create_model(optimizer="adam"):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=train_data[0].shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(len(class_names), activation="softmax")
    ])
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# Define the hyperparameters to tune
param_grid = {
    "optimizer": ["adam", "sgd"],
    "batch_size": [32, 64, 128],
    "epochs": [10, 20, 30]
}

# Create the model
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid_search.fit(train_data, train_labels)

# Print the best hyperparameters and accuracy
print("Best Hyperparameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)


<ipython-input-27-2026191f510d>:86: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model)


Epoch 1/10
3/3 [==============================] - 11s 2s/step - loss: 3.1909 - accuracy: 0.1711
Epoch 2/10
3/3 [==============================] - 9s 3s/step - loss: 1.9448 - accuracy: 0.2105
Epoch 3/10
3/3 [==============================] - 8s 2s/step - loss: 1.7505 - accuracy: 0.2368
Epoch 4/10
3/3 [==============================] - 10s 2s/step - loss: 1.7487 - accuracy: 0.1579
Epoch 5/10
3/3 [==============================] - 9s 3s/step - loss: 1.7098 - accuracy: 0.2500
Epoch 6/10
3/3 [==============================] - 8s 2s/step - loss: 1.7344 - accuracy: 0.2368
Epoch 7/10
3/3 [==============================] - 9s 2s/step - loss: 1.6683 - accuracy: 0.2368
Epoch 8/10
3/3 [==============================] - 9s 3s/step - loss: 1.6363 - accuracy: 0.2632
Epoch 9/10
3/3 [==============================] - 8s 2s/step - loss: 1.6022 - accuracy: 0.2500
Epoch 10/10
2/2 [==============================] - 1s 185ms/step - loss: 1.7866 - accuracy: 0.2564
Epoch 1/10
3/3 [============================

In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Set the path to your ZIP file
zip_file_path = '/content/Soil.zip'

# Set the class names
class_names = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy loam', 'Sandy soil']

# Set the desired image size
image_size = (224, 224)

# Create lists to store the data and labels
data = []
labels = []

# Read the images from the ZIP file and preprocess them
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    soil_types_folder = 'Soil Types'
    for class_name in class_names:
        class_dir = os.path.join(soil_types_folder, class_name)
        for file_name in zip_ref.namelist():
            if file_name.startswith(class_dir) and not file_name.endswith('/'):
                with zip_ref.open(file_name) as image_file:
                    image = Image.open(image_file)
                    # Resize the image to the desired size
                    image = image.resize(image_size)
                    # Convert the image to grayscale
                    image = image.convert('L')
                    # Perform any necessary additional preprocessing on the image here
                    # ...
                    # Convert the image to a NumPy array
                    image = np.array(image)
                    # Add the preprocessed image to the data list
                    data.append(image)
                    # Add the corresponding label to the labels list
                    labels.append(class_name)

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Add channel dimension to the data
data = np.expand_dims(data, axis=-1)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the pixel values to the range [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=train_data[0].shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(class_names), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_data, val_labels)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


In [ ]:
import os
import zipfile
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Set the path to your ZIP file
zip_file_path = '/content/Soil.zip'

# Set the class names
class_names = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy loam', 'Sandy soil']

# Set the desired image size
image_size = (224, 224)

# Create lists to store the data and labels
data = []
labels = []

# Read the images from the ZIP file and preprocess them
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    soil_types_folder = 'Soil Types'
    for class_name in class_names:
        class_dir = os.path.join(soil_types_folder, class_name)
        for file_name in zip_ref.namelist():
            if file_name.startswith(class_dir) and not file_name.endswith('/'):
                with zip_ref.open(file_name) as image_file:
                    image = Image.open(image_file)
                    # Resize the image to the desired size
                    image = image.resize(image_size)
                    # Convert the image to grayscale
                    image = image.convert('L')
                    # Perform any necessary additional preprocessing on the image here
                    # ...
                    # Convert the image to a NumPy array
                    image = np.array(image)
                    # Add the preprocessed image to the data list
                    data.append(image)
                    # Add the corresponding label to the labels list
                    labels.append(class_name)

# Convert the data and labels lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Add channel dimension to the data
data = np.expand_dims(data, axis=-1)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the pixel values to the range [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=train_data[0].shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(len(class_names), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_data, val_labels)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
zip_file_path = '/content/Soil.zip'
extract_path = '/content/Soil'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
train_dir = os.path.join(extract_path, 'Soil Types', 'train')
valid_dir = os.path.join(extract_path, 'Soil Types', 'valid')
test_dir = os.path.join(extract_path, 'Soil Types', 'test')



In [ ]:
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

for root, dirs, files in os.walk(extract_path):
    print(root, dirs, files)


/content/Soil ['Soil Types'] []
/content/Soil/Soil Types ['Sandy loam', 'Clayey soils', 'Sandy soil', 'Laterite soil', 'Loamy soil', 'Alluvial soil'] []
/content/Soil/Soil Types/Sandy loam [] ['sandy loam 1.png', 'sandy loam 23.png', 'sandy loam 18.png', 'sandy loam 10.png', 'sandy loam 2.png', 'sandy loam 16.png', 'sandy loam 12.png', 'sandy loam 5.png', 'sandy loam 26.png', 'sandy loam 9.png', 'sandy loam 7.png', 'sandy loam 21.png', 'sandy loam 24.png', 'sandy loam 3.png', 'sandy loam 15.png', 'sandy loam 20.png', 'sandy loam 6.png', 'sandy loam 11.png', 'sandy loam 13.png', 'sandy loam 17.png', 'sandy loam 22.png', 'sandy loam 25.png', 'sandy loam 19.png', 'sandy loam 8.png', 'sandy loam 4.png', 'sandy loam 14.png']
/content/Soil/Soil Types/Clayey soils [] ['clay 29.png', 'clay 6.png', 'clay 26.png', 'clay 13.png', 'clay 1.png', 'clay 5.png', 'clay 27.png', 'clay 11.png', 'clay 2.png', 'clay 10.png', 'clay 20.png', 'clay 28.png', 'clay 17.png', 'clay 22.png', 'clay 24.png', 'clay 1

In [ ]:
train_dir = os.path.join(extract_path, 'Soil Types', 'train')
valid_dir = os.path.join(extract_path, 'Soil Types', 'valid')
test_dir = os.path.join(extract_path, 'Soil Types', 'test')


In [ ]:
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
import glob

train_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types/*/*'))[0]
valid_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types/*/*'))[1]
test_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types/*/*'))[2]



In [ ]:
import glob

categories = ['Alluvial soil', 'Clayey soils', 'Laterite soil', 'Loamy soil', 'Sandy soil', 'Sandy loam']

train_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types', categories[0], '*'))
valid_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types', categories[1], '*'))
test_dir = glob.glob(os.path.join(extract_path, 'Soil/Soil Types', categories[2], '*'))


In [ ]:
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


In [ ]:
model.save("Soil_model.h5")